In [1]:
import sys
import os
import pandas as pd
import json
import numpy as np
import pickle
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer
from datasets import load_from_disk, load_metric, Dataset, load_dataset

#from captum.concept import TCAV, Concept
from captum.concept._utils.data_iterator import dataset_to_dataloader, CustomIterableDataset


In [2]:
sys.path.insert(0,'/zhome/94/5/127021/speciale/master_project')
from src.models.captum.tcav import TCAV
from src.models.captum.concept import Concept

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
checkpoint = '/work3/s174498/final/linear_head/checkpoint-1500' #finetuning-sentiment-model-test-head3/checkpoint-500'

model = RobertaForSequenceClassification.from_pretrained(checkpoint,output_hidden_states = True,return_dict = True,)
#model.eval()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /work3/s174498/final/linear_head/checkpoint-1500 and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# tokenizer
tokenizer_checkpoint = RobertaTokenizer.from_pretrained(checkpoint) 
#tokenizer_pretrained = RobertaTokenizer.from_pretrained('roberta-base')

In [6]:
# load
datadir = '/work3/s174498/sst2_dataset/'
#test_0_dataset = load_from_disk(datadir + 'test_0_dataset')
test_1_dataset = load_from_disk(datadir + 'test_1_dataset')

In [7]:
#tokenizer_checkpoint.encode(test_0_dataset['sentence'][0])

In [8]:
#model.roberta.encoder.layer[0].attention

In [9]:
N = 200
#for i in range(len(test_1_dataset['sentence'])):
#    j = len(test_1_dataset['sentence'][i])
#    if j > N :
#        N = j



In [10]:
def get_tensor_from_filename(filename):
    #datadir = '/work3/s174498/sst2_dataset/'
    ds = load_from_disk(filename)
    const_len = N
    ds_text = ds['sentence']

    for i in range(len(ds_text)):
        #text_idx = tokenizer_checkpoint.tokenize(ds_text[i])
        #text_idx[i] = tokenizer_checkpoint.encode(text_idx[i])
        text_idx = tokenizer_checkpoint.encode(ds_text[i])
        text_idx += [1] * max(0, const_len - len(text_idx[i]))
        text_indices = torch.tensor(text_idx, device=device).unsqueeze(0)
        yield text_indices

## Hent concept data

In [11]:
def assemble_concept(name, id, concepts_path="/work3/s174498/sst2_dataset/"):
    concept_path = os.path.join(concepts_path, name) + "/"
    #print(concept_path)
    dataset = CustomIterableDataset(get_tensor_from_filename, concept_path)
    #print(dataset.)
    concept_iter = dataset_to_dataloader(dataset, batch_size=1)
    #print(concept_iter)

    return Concept(id=id, name=name, data_iter=concept_iter)

In [12]:
pos_concept = assemble_concept("positive", 0, concepts_path="/work3/s174498/sst2_dataset/")
#pos_concept2 = assemble_concept("positive", 2, concepts_path="/work3/s174498/sst2_dataset/")

neg_concept = assemble_concept("negative", 5, concepts_path="/work3/s174498/sst2_dataset/")

In [13]:
experimental_sets=[[pos_concept, neg_concept]]

### Input target text
**Should be converted to tensor**

In [14]:
input_text = test_1_dataset['sentence'][:10]

In [15]:
def covert_text_to_tensor(input_texts):
    input_tensors = []
    for input_text in input_texts:
        const_len = N
        #text_idx = tokenizer_checkpoint.tokenize(input_text)
        text_idx = tokenizer_checkpoint.encode(input_text)
        text_idx += [1] * max(0, const_len - len(text_idx))
        input_tensor = torch.tensor(text_idx, device=device).unsqueeze(0)
        input_tensors.append(input_tensor)
    return torch.cat(input_tensors)

input_text_indices = covert_text_to_tensor(input_text)


In [16]:
tcav = TCAV(model, layers=['roberta.encoder.layer.0.output.LayerNorm'])

/zhome/94/5/127021/miniconda3/envs/roberta_env/lib/python3.9/site-packages/captum/concept/_utils/classifier.py:130: UserWarning: Using default classifier for TCAV which keeps input both train and test datasets in the memory. Consider defining your own classifier that doesn't rely heavily on memory, for large number of concepts, by extending `Classifer` abstract class
  warnings.warn(


In [17]:
tcav.interpret(input_text_indices , experimental_sets=experimental_sets)

>>> TEST 1 <<<
>>> CAVs 1
>>> CAVs 2.0
>>> CAVs 2.1
>>> gen.act. 1
>>> GEN ACT 1
>>> GEN ACT 2
>>> GEN ACT 3
>>> GEN ACT 4


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'generator'>

In [19]:
pos_concept.data_iter

TypeError: 'DataLoader' object is not subscriptable

In [ ]:
model.roberta.encoder#.layer[0].attention

: 

: 